This notebook will mainly be used for the IBM Data Science Capstone Project by IBM and Coursera

In [1]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Introduction
For this project, I will be comparing neighborhoods clusters in Singapore to those in New York and Toronto to see which are they more similar to. I would ultimately like to answer the question of whether Singapore is more similar to New York or Toronto. 

## Business Problem
The business problem I would like to solve is a hypothetical one. Imagine that there is a Singaporean franchise looking to expand its business overseas, either to New York or Toronto. This company has hired me to determine which neighborhood or neighborhoods is most similar to their most successful branch in Singapore which is located the neighborhood of Orchard. This information will help them determine 1) which country to enter and 2) where to set up their new store or potentially stores. 

## Data
Firstly, I will be gathering the postal codes of neighbors and using the geopy to determine their latitude and longitud. After this, foursquare data can be retrieved and used to compare the neighborhoods. Foursquare data in this situation is sufficient to cluster and segment neighborhoods in the three countries using k-means clustering. Individual neighborhood in the different countries can also be compared through a similarity matrix where their scores are based on the frequency of venues. The country to enter would be determined by the size of the similar cluster as well as the similarity of the cluster to their branch in Singapore.
